'pipinstall' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
%pip install streamlit plotly

import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# Page config
st.set_page_config(page_title="ATL Carbon Dashboard", layout="wide", page_icon="🌳")

# Title
st.title("🌳 Hartsfield-Jackson Atlanta Airport Carbon Sequestration Dashboard")
st.markdown("*Conceptual Framework for Stakeholder Engagement*")

# Sidebar for navigation
module = st.sidebar.radio(
    "Select Module",
    ["Overview", "Buildings", "Transportation", "Airlines/SAF", "Urban Forest"]
)

# Sample data
def get_sample_data():
    return {
        'buildings': {'emissions': 45000, 'stakeholders': ['Airport Authority', 'Facility Managers', 'Energy Providers']},
        'transportation': {'emissions': 38000, 'stakeholders': ['MARTA', 'Ride-share Companies', 'Parking Operators']},
        'airlines': {'emissions': 520000, 'stakeholders': ['Delta Air Lines', 'SAF Suppliers', 'FAA']},
        'urban_forest': {'sequestration': 12500, 'stakeholders': ['City of Atlanta Parks', 'Tree Atlanta', 'Airport Landscaping']}
    }

data = get_sample_data()

# Calculate net balance
total_emissions = data['buildings']['emissions'] + data['transportation']['emissions'] + data['airlines']['emissions']
total_sequestration = data['urban_forest']['sequestration']
net_balance = total_emissions - total_sequestration
offset_percentage = (total_sequestration / total_emissions * 100)

# Overview Module
if module == "Overview":
    st.header("Carbon Balance Overview")

    col1, col2, col3 = st.columns(3)
    col1.metric("Total Emissions", f"{total_emissions:,} tCO₂e/year")
    col2.metric("Urban Forest Sequestration", f"{total_sequestration:,} tCO₂e/year", delta=f"-{total_sequestration:,}")
    col3.metric("Net Balance", f"{net_balance:,} tCO₂e/year", delta=f"-{offset_percentage:.1f}% offset")

    # Sankey diagram
    st.subheader("Carbon Flow")
    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = ["Buildings", "Transportation", "Airlines", "Total Emissions", "Urban Forest", "Net Balance"],
            color = ["#e74c3c", "#e67e22", "#c0392b", "#95a5a6", "#27ae60", "#34495e"]
        ),
        link = dict(
            source = [0, 1, 2, 3, 4],
            target = [3, 3, 3, 5, 5],
            value = [
                data['buildings']['emissions'],
                data['transportation']['emissions'],
                data['airlines']['emissions'],
                total_emissions,
                total_sequestration
            ]
        )
    )])
    fig.update_layout(height=400)
    st.plotly_chart(fig, use_container_width=True)

    # All stakeholders
    st.subheader("Key Stakeholder Groups")
    all_stakeholders = []
    for key in data:
        all_stakeholders.extend(data[key]['stakeholders'])

    cols = st.columns(4)
    for i, stakeholder in enumerate(all_stakeholders):
        cols[i % 4].markdown(f"• {stakeholder}")

# Buildings Module
elif module == "Buildings":
    st.header("🏢 Buildings Module")

    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Operational Emissions")
        st.metric("Annual Emissions", f"{data['buildings']['emissions']:,} tCO₂e/year")

        # Sample breakdown
        building_data = pd.DataFrame({
            'Category': ['Terminals', 'Concourses', 'Support Facilities', 'Parking'],
            'Emissions': [18000, 15000, 8000, 4000]
        })

        fig = px.bar(building_data, x='Category', y='Emissions',
                     title='Emissions by Building Type',
                     color='Emissions',
                     color_continuous_scale='Reds')
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        st.subheader("Stakeholder Engagement")
        for stakeholder in data['buildings']['stakeholders']:
            st.markdown(f"**{stakeholder}**")
            st.progress(0.7)
            st.caption("Active collaboration")

        st.markdown("---")
        st.markdown("**Key Actions:**")
        st.markdown("• Energy audits")
        st.markdown("• LED retrofits")
        st.markdown("• Solar installations")

# Transportation Module
elif module == "Transportation":
    st.header("🚗 Ground Transportation Module")

    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Transportation Emissions")
        st.metric("Annual Emissions", f"{data['transportation']['emissions']:,} tCO₂e/year")

        # Sample mode breakdown
        transport_data = pd.DataFrame({
            'Mode': ['Personal Vehicles', 'Ride-share', 'Taxis', 'MARTA', 'Shuttles'],
            'Emissions': [20000, 9000, 5000, 2000, 2000],
            'Passengers': [15000000, 8000000, 3000000, 5000000, 2000000]
        })

        fig = px.bar(transport_data, x='Mode', y='Emissions',
                     title='Emissions by Transportation Mode',
                     color='Emissions',
                     color_continuous_scale='Oranges')
        fig.update_layout(xaxis_tickangle=-45)
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        st.subheader("Stakeholder Engagement")
        for stakeholder in data['transportation']['stakeholders']:
            st.markdown(f"**{stakeholder}**")
            st.progress(0.6)
            st.caption("Moderate engagement")

        st.markdown("---")
        st.markdown("**Key Actions:**")
        st.markdown("• EV charging stations")
        st.markdown("• Transit incentives")
        st.markdown("• Carpool programs")

# Airlines/SAF Module
elif module == "Airlines/SAF":
    st.header("✈️ Airlines & Sustainable Aviation Fuel Module")

    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Aviation Emissions")
        st.metric("Annual Emissions", f"{data['airlines']['emissions']:,} tCO₂e/year")

        # SAF adoption scenario
        years = [2024, 2025, 2026, 2027, 2028, 2030]
        baseline = [520000] * 6
        saf_5 = [520000, 510000, 495000, 475000, 455000, 420000]
        saf_20 = [520000, 500000, 465000, 425000, 380000, 320000]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=years, y=baseline, name='Baseline (No SAF)', line=dict(color='red', dash='dash')))
        fig.add_trace(go.Scatter(x=years, y=saf_5, name='5% SAF Adoption', line=dict(color='orange')))
        fig.add_trace(go.Scatter(x=years, y=saf_20, name='20% SAF Adoption', line=dict(color='green')))

        fig.update_layout(title='SAF Adoption Scenarios',
                         xaxis_title='Year',
                         yaxis_title='Emissions (tCO₂e/year)')
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        st.subheader("Stakeholder Engagement")
        for stakeholder in data['airlines']['stakeholders']:
            st.markdown(f"**{stakeholder}**")
            st.progress(0.5)
            st.caption("Building partnerships")

        st.markdown("---")
        st.markdown("**Key Actions:**")
        st.markdown("• SAF procurement")
        st.markdown("• Fleet modernization")
        st.markdown("• Carbon offsets")

# Urban Forest Module
elif module == "Urban Forest":
    st.header("🌳 Urban Forest Sequestration Module")

    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Carbon Sequestration")
        st.metric("Annual Sequestration", f"{data['urban_forest']['sequestration']:,} tCO₂e/year", delta="8%", delta_color="normal")

        # Sample tree species distribution
        species_data = pd.DataFrame({
            'Species': ['Loblolly Pine', 'Southern Red Oak', 'Sweetgum', 'Willow Oak', 'Other'],
            'Count': [850, 620, 480, 390, 1160],
            'Sequestration': [4200, 3100, 2100, 1800, 1300]
        })

        fig = px.scatter(species_data, x='Count', y='Sequestration',
                        size='Sequestration', color='Species',
                        title='Tree Species Distribution & Sequestration Rates',
                        labels={'Count': 'Number of Trees', 'Sequestration': 'Annual CO₂ Sequestration (tCO₂e)'})
        st.plotly_chart(fig, use_container_width=True)

        # Simple map placeholder
        st.subheader("Spatial Distribution")
        map_data = pd.DataFrame({
            'lat': [33.6407 + i*0.01 for i in range(-3, 4)],
            'lon': [-84.4277 + i*0.01 for i in range(-3, 4)],
            'sequestration': [1500, 2000, 2500, 3000, 2500, 2000, 1500]
        })
        st.map(map_data)

    with col2:
        st.subheader("Stakeholder Engagement")
        for stakeholder in data['urban_forest']['stakeholders']:
            st.markdown(f"**{stakeholder}**")
            st.progress(0.8)
            st.caption("Strong partnership")

        st.markdown("---")
        st.markdown("**Key Actions:**")
        st.markdown("• Tree planting campaigns")
        st.markdown("• Canopy monitoring")
        st.markdown("• Species diversification")
        st.markdown("• Green infrastructure")

# Footer
st.markdown("---")
st.caption(f"Last updated: {datetime.now().strftime('%B %d, %Y')} | Conceptual Framework | Georgia Tech - Sustainable Urban Systems Lab")


2025-12-04 09:13:23.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.874 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.875 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 09:13:23.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Note: you may need to restart the kernel to use updated packages.


DeltaGenerator()